In [ ]:
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import pydicom

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys.labs.autosegmentation import pipeline, filtering, indexing

In [ ]:
# Put all of the DICOM data within a directory called 'dicom' 
# organised by 'training', 'validation', and 'testing' in here:
data_path_root = pathlib.Path.home().joinpath('.data/dicom-ct-and-structures')

# Of note, the DICOM file directory structure need not have any further
# organisation beyond being placed somewhere within one of the three
# 'training', 'validation', or 'testing'. They can be organised into
# directories by patient but that is not a requirement.

In [ ]:
# The following names_map is used to standardise the structure names
names_map = filtering.load_names_mapping('name_mappings.json')

In [ ]:
# Create masks for the following structures, in the following order
structures_to_learn = [
    'lens_left', 'lens_right', 'eye_left', 'eye_right', 'patient']

# Use the following to filter the slices used for training, validation, and testing
filters = {
    "study_set_must_have_all_of": structures_to_learn,
    "slice_at_least_one_of": ['lens_left', 'lens_right', 'eye_left', 'eye_right'],
    "slice_must_have": ['patient'],
    "slice_cannot_have": []
}

In [ ]:
datasets = pipeline.create_datasets(data_path_root, names_map, structures_to_learn, filters)

In [ ]:
(
    ct_image_paths,
    structure_set_paths,
    ct_uid_to_structure_uid,
    structure_uid_to_ct_uids,
) = indexing.get_uid_cache(data_path_root)

In [ ]:
(
    structure_names_by_ct_uid,
    structure_names_by_structure_set_uid,
) = indexing.get_cached_structure_names_by_uids(
    data_path_root, structure_set_paths, names_map
)

In [ ]:
def get_path_from_tensor_uid(uid):
    return ct_image_paths[uid.numpy().decode()]

In [ ]:
ct_uid, x_grid, y_grid, input_array, output_array = list(datasets['training'].take(1))[-1]

ct_uid = ct_uid.numpy().decode()
x_grid = x_grid.numpy()
y_grid = y_grid.numpy()
input_array = input_array.numpy()[:,:,0]
output_array = output_array.numpy()

In [ ]:
structure_set_uid = ct_uid_to_structure_uid[ct_uid]
structure_set_uid

In [ ]:
structure_set_path = structure_set_paths[structure_set_uid]
structure_set = pydicom.read_file(
    structure_set_path,
    force=True,
    specific_tags=["ROIContourSequence", "StructureSetROISequence"],
)

In [ ]:
number_to_name_map = {
    roi_sequence_item.ROINumber: names_map[roi_sequence_item.ROIName]
    for roi_sequence_item in structure_set.StructureSetROISequence
    if names_map[roi_sequence_item.ROIName] is not None
}

number_to_name_map

In [ ]:
structure_set.StructureSetROISequence[10]

In [ ]:
structure_names_by_structure_set_uid[structure_set_uid]

In [ ]:
for roi_contour_sequence_item in structure_set.ROIContourSequence:
    try:
        structure_name = number_to_name_map[
            roi_contour_sequence_item.ReferencedROINumber
        ]
    except KeyError:
        continue
        
    for contour_sequence_item in roi_contour_sequence_item.ContourSequence:
        contour_imaging_sequence = contour_sequence_item.ContourImageSequence
        referenced_ct_uid = contour_imaging_sequence[0].ReferencedSOPInstanceUID
        
        if referenced_ct_uid == ct_uid:
            print(structure_name)
#             print(contour_sequence_item.ContourData)

In [ ]:
structure_names_by_ct_uid[ct_uid]

In [ ]:
plt.contourf(x_grid, y_grid, input_array)
plt.axis('equal')

In [ ]:
for i, structure in enumerate(structures_to_learn):
    plt.figure()
    plt.contourf(x_grid, y_grid, output_array[:,:,i])
    plt.title(structure)
    plt.axis('equal')
    plt.show()
    
    print(np.max(output_array[:,:,i]))